---
title: JUNO Magnetic field data pipeline
---

JUNO Magnetic field data can be downloaded from [PDS](https://pds-ppi.igpp.ucla.edu/mission/JUNO/JNO/FGM) website.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp missions/juno/fgm
# | hide
# | export
import os
import polars as pl
from zipfile import ZipFile
import pooch
from pooch.processors import Unzip

from pipe import filter
from space_analysis.utils.lbl import load_lbl

from typing import Literal, Callable
from functools import partial

```txt
File Naming Convention                                                        
==============================================================================
Convention:                                                                   
   fgm_jno_LL_CCYYDDDxx_vVV.ext                                               
Where:                                                                        
   fgm - Fluxgate Magnetometer three character instrument abbreviation        
   jno - Juno                                                                 
    LL - CODMAC Data level, for example, l3 for level 3                       
    CC - The century portion of a date, 20                                    
    YY - The year of century portion of a date, 00-99                         
   DDD - The day of year, 001-366                                             
    xx - Coordinate system of data (se = Solar equatorial, ser = Solar        
         equatorial resampled, pc = Planetocentric, ss = Sun-State,           
         pl = Payload)                                                        
     v - separator to denote Version number                                   
    VV - version                                                              
   ext - file extension (sts = Standard Time Series (ASCII) file, lbl = Label 
         file)                                                                
Example:                                                                      
   fgm_jno_l3_2014055se_v00.sts    
```

## Downloading data 

In [ ]:
# | code-summary: type definitions
# | export
JunoPhases = Literal["CRUISE", "JUPITER"]
JunoFGMCoords = Literal["SE", "SS", "PL"]
JunoFGMTimeResolutions = Literal["1SEC", "1MIN", "FULL"]

In [ ]:
# | exporti
def unzip_convert_lbl(member: str, fname: str, extract_dir, load_func: Callable, clean=True, fmt="arrow"):
    with ZipFile(fname, "r") as zip_file:
        lbl_fp = zip_file.extract(member, path=extract_dir)
        sts_fp = zip_file.extract(member.replace(".lbl", ".sts"), path=extract_dir)

        # Convert the file to a different format
        df = load_func(lbl_fp).collect()
        output_fp = lbl_fp.replace("lbl", fmt)
        df.write_ipc(output_fp)

        # Remove the lbl and sts files
        if clean:
            os.remove(lbl_fp)
            os.remove(sts_fp)

        return output_fp

In [ ]:
# | export
def load_jno_lbl(file: str):
    df: pl.DataFrame = pl.from_dataframe(load_lbl(file))
    return (
        df.lazy()
        .with_columns(
            time=pl.col("SAMPLE UTC").str.slice(0, 4).str.to_datetime("%Y")
            + pl.duration(
                milliseconds=(pl.col("DECIMAL DAY") - 1) * 24 * 60 * 60 * 1000
            )
        )
        .drop(["SAMPLE UTC", "DECIMAL DAY", "INSTRUMENT RANGE", "X", "Y", "Z"])
        .sort("time")
    )

def unpack_and_convert(fname, extract_dir, process_func=unzip_convert_lbl):
    """
    Post-processing hook to unzip a file and convert it to a different format in real-time. (Otherwise the files unzipped would take up too much space on the user's computer.)

    Parameters
    ----------
    fname : str
       Full path of the zipped file in local storage

    """

    with ZipFile(fname, "r") as zip_file:
        # Extract the data file from within the archive
        members = list(zip_file.namelist() | filter(lambda x: x.endswith(".lbl")))

    func = partial(process_func, fname=fname, extract_dir=extract_dir, load_func=load_jno_lbl)
    futures = list(map(func, members))

    return futures


class UnpackConvert(Unzip):
    
    old_fmt = "lbl"
    new_fmt = "arrow"

    def _extract_file(self, fname, extract_dir):
        unpack_and_convert(fname, extract_dir)
        
    def _all_members(self, fname):
        """Return all members from a given archive."""
        with ZipFile(fname, "r") as zip_file:
            files = zip_file.namelist()
            oldfiles = [f for f in files if f.endswith(self.old_fmt)]
            return [f.replace(self.old_fmt, self.new_fmt) for f in oldfiles]

In [ ]:
# | export

PDS_URL_FMT = "https://pds-ppi.igpp.ucla.edu/ditdos/download?id=pds://PPI/{dataset}/DATA/{phase}/{coord}/{datatype}"

def download_data(
    dataset="JNO-SS-3-FGM-CAL-V1.0",
    phase: JunoPhases = "CRUISE",
    coord: JunoFGMCoords = "SE",
    datatype: JunoFGMTimeResolutions = "1SEC",  # time resolution
    processor: Callable = None,
    url_fmt=PDS_URL_FMT,
    fmt="arrow",
) -> list[str]:
    
    url = url_fmt.format(dataset=dataset, phase=phase, coord=coord, datatype=datatype)

    if processor is None:
        processor = UnpackConvert()  # default processor, needed to be created here!!!

    files = pooch.retrieve(
        url=url,
        known_hash=None,
        progressbar=True,
        processor=processor,
    )

    return sorted(files | filter(lambda x: x.endswith(f".{fmt}")))

In [ ]:
download_data(datatype="FULL")
# download_data()

In [ ]:
# | hide
from nbdev import nbdev_export

nbdev_export()

## Dataset Overview

### Index

In [ ]:
pds_dir = "https://pds-ppi.igpp.ucla.edu/data"

possible_coords = ["se", "ser", "pc", "ss", "pl"]
possible_exts = ["sts", "lbl"]
possible_data_rates = ["1s", "1min", "1h"]

juno_ss_config = {
    "DATA_SET_ID": "JNO-SS-3-FGM-CAL-V1.0",
    "FILE_SPECIFICATION_NAME": "INDEX/INDEX.LBL",
}

juno_j_config = {
    "DATA_SET_ID": "JNO-J-3-FGM-CAL-V1.0",
    "FILE_SPECIFICATION_NAME": "INDEX/INDEX.LBL",
}

#### Process index

In [ ]:
# | eval: false
import pandas
import pdpipe as pdp

In [ ]:
# | eval: false
def process_jno_index(df: pandas.DataFrame):
    _index_time_format = "%Y-%jT%H:%M:%S.%f"

    df.columns = df.columns.str.replace(" ", "")
    jno_index_pipeline = pdp.PdPipeline(
        [
            pdp.ColDrop(["PRODUCT_ID", "CR_DATE", "PRODUCT_LABEL_MD5CHECKSUM"]),
            pdp.ApplyByCols("SID", str.rstrip),
            pdp.ApplyByCols("FILE_SPECIFICATION_NAME", str.rstrip),
            pdp.ColByFrameFunc(
                "START_TIME",
                lambda df: pandas.to_datetime(
                    df["START_TIME"], format=_index_time_format
                ),
            ),
            pdp.ColByFrameFunc(
                "STOP_TIME",
                lambda df: pandas.to_datetime(
                    df["STOP_TIME"], format=_index_time_format
                ),
            ),
            # pdp.ApplyByCols(['START_TIME', 'STOP_TIME'], pandas.to_datetime, format=_index_time_format), # NOTE: This is slow
        ]
    )

    return jno_index_pipeline(df)